In [ ]:
# Imports
import numpy as np
import spikeextractors as se

In [ ]:
# Properties of the in-memory dataset
num_channels=7
samplerate=30000
duration=20
num_timepoints=int(samplerate*duration)
num_units=4
num_events=1000

In [ ]:
# Generate a pure-noise timeseries dataset and a linear geometry
timeseries=np.random.normal(0,10,(num_channels,num_timepoints))
geom=np.zeros((num_channels,2))
geom[:,0]=range(num_channels)

# Define the in-memory recording extractor
RX=se.NumpyRecordingExtractor(timeseries=timeseries,geom=geom,samplerate=samplerate)

# Generate some random events
times=np.int_(np.sort(np.random.uniform(0,num_timepoints,num_events)))
labels=np.random.randint(1,num_units+1,size=num_events)
    
# Define the in-memory sorting extractor
SX=se.NumpySortingExtractor()
for k in range(1,num_units+1):
    times_k=times[np.where(labels==k)[0]]
    SX.add_unit(unit_id=k,times=times_k)

In [ ]:
# Demonstrate the API for extracting information
print('Unit ids = {}'.format(SX.get_unit_ids()))
st=SX.get_unit_spike_train(unit_id=1)
print('Num. events for unit 1 = {}'.format(len(st)))
st1=SX.get_unit_spike_train(unit_id=1,start_frame=0,end_frame=30000)
print('Num. events for first second of unit 1 = {}'.format(len(st1)))

In [ ]:
# Write the input/output in the MountainSort format
se.MdaRecordingExtractor.write_recording(recording=RX,save_path='sample_mountainsort_dataset')
se.MdaSortingExtractor.write_sorting(sorting=SX,save_path='sample_mountainsort_dataset/firings_true.mda')

In [ ]:
# Read this dataset with the Mda input extractor
RX2=se.MdaRecordingExtractor(dataset_directory='sample_mountainsort_dataset')
SX2=se.MdaSortingExtractor(firings_file='sample_mountainsort_dataset/firings_true.mda')

In [ ]:
# We should get he same information as above
print('Unit ids = {}'.format(SX2.get_unit_ids()))
st=SX2.get_unit_spike_train(unit_id=1)
print('Num. events for unit 1 = {}'.format(len(st)))
st1=SX2.get_unit_spike_train(unit_id=1,start_frame=0,end_frame=30000)
print('Num. events for first second of unit 1 = {}'.format(len(st1)))

In [ ]:
#Calculate a unit property and store it in my SortingExtractor
full_spike_train = SX2.get_unit_spike_train(unit_id=1)
firing_rate = float(len(full_spike_train))/RX2.get_num_frames()
SX2.set_unit_property(unit_id=1, property_name='firing_rate', value=firing_rate)
print('Average firing rate during the recording of unit 1 = {}'.format(SX2.get_unit_property(unit_id=1, property_name='firing_rate')))

In [ ]:
#Can get a the sub-dataset from the sorting
SX3=se.SubSortingExtractor(parent_sorting=SX2,unit_ids=[1, 2],
                           start_frame=10000,end_frame=20000)

In [ ]:
print('Num. units = {}'.format(len(SX3.get_unit_ids())))
print('Average firing rate of units1 during frames 14000-16000 = {}'.format(
                        float(len(SX3.get_unit_spike_train(unit_id=1)))/6000))